In [1]:
import os
os.chdir(path="/labs/gevaertlab/users/yyhhli/code/vae")
from applications.application import Application
import scipy
import pandas as pd
from tqdm import tqdm
from statsmodels.stats.multitest import fdrcorrection
from IPython.display import display, HTML


In [3]:
task_names = ['volume', 'malignancy', 'texture', 'spiculation', 'subtlety']

In [4]:
app = Application(log_name='VAE3D32AUG',
                  version=10,
                  task_name='volume')

match labels | 0.0 secs.


In [5]:
# load data
X, Y = app.preprocess_data()

In [6]:
print(X['train'].shape)
print(Y['train'].shape)

(5505, 2048)
(5505,)


In [7]:
spmr = scipy.stats.spearmanr(X['train'][:,0], Y['train'], axis=0)

In [8]:
list(spmr)[1].shape

()

In [13]:
def get_stats_results(X, Y, func):
    results = []
    for col in tqdm(range(X.shape[1])):
        results.append(list(func(X[:,col], Y)))
    return pd.DataFrame(results, columns=['correlation', 'pvalue'])

In [14]:
spm_df = get_stats_results(X['train'], Y['train'], scipy.stats.spearmanr)

100%|██████████| 2048/2048 [00:03<00:00, 578.83it/s]


In [15]:
spm_df.head()

,correlation,pvalue
0,0.002649,8.442033e-01
1,0.007467,5.796621e-01
2,-0.063629,2.304361e-06
3,0.005750,6.697374e-01
4,-0.141261,6.191126e-26


In [20]:
fdr_df = pd.DataFrame(fdrcorrection(pvals=list(spm_df['pvalue']))).transpose()

In [23]:
fdr_df.columns = ['reject', 'adjusted']

In [24]:
fdr_df

,reject,adjusted
0,False,0.875407
1,False,0.642049
2,True,1.03042e-05
3,False,0.72611
4,True,1.69059e-24
...,...,...
2043,True,0.000473252
2044,False,0.0638369
2045,False,0.0874867
2046,True,0.0186235


In [25]:
adj_spm_df = pd.concat([spm_df, fdr_df], axis=1)

In [26]:
adj_spm_df

,correlation,pvalue,reject,adjusted
0,0.002649,8.442033e-01,False,0.875407
1,0.007467,5.796621e-01,False,0.642049
2,-0.063629,2.304361e-06,True,1.03042e-05
3,0.005750,6.697374e-01,False,0.72611
4,-0.141261,6.191126e-26,True,1.69059e-24
...,...,...,...,...
2043,-0.050816,1.619871e-04,True,0.000473252
2044,-0.027338,4.252909e-02,False,0.0638369
2045,-0.025273,6.078785e-02,False,0.0874867
2046,-0.034666,1.010286e-02,True,0.0186235


In [27]:
associated_df = adj_spm_df[adj_spm_df['reject'] == True]

In [28]:
associated_df

,correlation,pvalue,reject,adjusted
2,-0.063629,2.304361e-06,True,1.03042e-05
4,-0.141261,6.191126e-26,True,1.69059e-24
5,0.115091,1.073420e-17,True,1.71747e-16
6,0.078188,6.292989e-09,True,4.06563e-08
7,0.085471,2.128383e-10,True,1.68298e-09
...,...,...,...,...
2033,-0.032193,1.691153e-02,True,0.0289589
2035,-0.032240,1.674877e-02,True,0.0287322
2042,-0.089355,3.104901e-11,True,2.69442e-10
2043,-0.050816,1.619871e-04,True,0.000473252


## Associations

In [45]:
task_names = ['volume', 'malignancy', 'texture', 'spiculation', 'subtlety']
def get_associated_features(task_names, func):
    sig_dict = {}
    for task_name in task_names:
        print(f"---start task {task_name}---")
        app = Application(log_name='VAE3D32AUG',
                    version=10,
                    task_name=task_name)
        X, Y = app.preprocess_data()
        print(X['train'].shape, len(Y['train']))
        stats_df = get_stats_results(X['train'], Y['train'], func)
        # correction
        fdr_df = pd.DataFrame(fdrcorrection(pvals=list(stats_df['pvalue']))).transpose()
        fdr_df.columns = ['reject', 'adjusted']
        adj_stats_df = pd.concat([stats_df, fdr_df], axis=1)
        sig_df = adj_stats_df[adj_stats_df['reject'] == True]
        sig_dict[task_name] = sig_df
    return sig_dict

In [ ]:
task_names = ['volume', 'malignancy', 'texture', 'spiculation', 'subtlety']
def get_associated_features(task_names, func):
    sig_dict = {}
    for task_name in task_names:
        print(f"---start task {task_name}---")
        app = Application(log_name='VAE3D32AUG',
                    version=10,
                    task_name=task_name)
        X, Y = app.preprocess_data()
        print(X['train'].shape, len(Y['train']))
        stats_df = get_stats_results(X['train'], Y['train'], func)
        # correction
        fdr_df = pd.DataFrame(fdrcorrection(pvals=list(stats_df['pvalue']))).transpose()
        fdr_df.columns = ['reject', 'adjusted']
        adj_stats_df = pd.concat([stats_df, fdr_df], axis=1)
        sig_df = adj_stats_df[adj_stats_df['reject'] == True]
        sig_dict[task_name] = sig_df
    return sig_dict

In [44]:
func = scipy.stats.spearmanr
for task_name in task_names:
    print(f"---start task {task_name}---")
    app = Application(log_name='VAE3D32AUG',
                version=10,
                task_name=task_name)
    X, Y = app.preprocess_data()
    print(X['train'].shape, len(Y['train']))
    stats_df = get_stats_results(X['train'], Y['train'], func)
    # correction
    fdr_df = pd.DataFrame(fdrcorrection(pvals=list(stats_df['pvalue']))).transpose()
    fdr_df.columns = ['reject', 'adjusted']
    adj_stats_df = pd.concat([stats_df, fdr_df], axis=1)
    sig_df = adj_stats_df[adj_stats_df['reject'] == True]

---start task volume---
match labels | 0.0 secs.


  3%|▎         | 62/2048 [00:00<00:03, 617.00it/s]

(5505, 2048) 5505


100%|██████████| 2048/2048 [00:03<00:00, 619.15it/s]


---start task malignancy---
match labels | 0.0 secs.


  3%|▎         | 60/2048 [00:00<00:03, 590.98it/s]

(5505, 2048) 5505


 59%|█████▊    | 1200/2048 [00:02<00:01, 586.40it/s]


KeyboardInterrupt: 

In [47]:
d = get_associated_features(task_names, func)

---start task volume---
match labels | 0.0 secs.


  3%|▎         | 62/2048 [00:00<00:03, 617.93it/s]

(5505, 2048) 5505


100%|██████████| 2048/2048 [00:03<00:00, 621.90it/s]


---start task malignancy---
match labels | 0.0 secs.


...

100%|██████████| 2048/2048 [00:03<00:00, 586.04it/s]


---start task subtlety---
match labels | 0.0 secs.


  2%|▏         | 38/2048 [00:00<00:05, 376.53it/s]

(5505, 2048) 5505


100%|██████████| 2048/2048 [00:04<00:00, 442.59it/s]


In [55]:
for k, df in d.items():
    display(df)
    print(f"{k}: {len(df)}")

,correlation,pvalue,reject,adjusted
2,-0.063629,2.304361e-06,True,1.03042e-05
4,-0.141261,6.191126e-26,True,1.69059e-24
5,0.115091,1.073420e-17,True,1.71747e-16
6,0.078188,6.292989e-09,True,4.06563e-08
7,0.085471,2.128383e-10,True,1.68298e-09
...,...,...,...,...
2033,-0.032193,1.691153e-02,True,0.0289589
2035,-0.032240,1.674877e-02,True,0.0287322
2042,-0.089355,3.104901e-11,True,2.69442e-10
2043,-0.050816,1.619871e-04,True,0.000473252


volume: 1321


,correlation,pvalue,reject,adjusted
0,-0.034244,1.105637e-02,True,0.0405509
1,0.067450,5.470046e-07,True,6.0884e-06
4,-0.117590,2.076766e-18,True,1.25095e-16
5,0.048106,3.562555e-04,True,0.00203452
6,0.074475,3.163171e-08,True,4.83446e-07
...,...,...,...,...
1970,-0.034322,1.087417e-02,True,0.0400545
1997,-0.060135,8.014726e-06,True,6.89671e-05
2005,-0.034769,9.882709e-03,True,0.0370014
2011,-0.041934,1.858243e-03,True,0.00874869


malignancy: 582


,correlation,pvalue,reject,adjusted
1,-0.047618,4.089390e-04,True,0.000749112
5,0.049493,2.391940e-04,True,0.00044252
9,-0.050341,1.865325e-04,True,0.000347922
11,-0.030785,2.236356e-02,True,0.0353399
14,-0.083719,4.935731e-10,True,1.34778e-08
...,...,...,...,...
2043,-0.056203,3.014541e-05,True,5.93633e-05
2044,-0.040585,2.597299e-03,True,0.00451551
2045,-0.071259,1.205646e-07,True,5.00844e-07
2046,-0.070442,1.678635e-07,True,6.22798e-07


texture: 1327


,correlation,pvalue,reject,adjusted
4,-0.101165,5.329240e-14,True,3.76355e-12
7,0.034967,9.469459e-03,True,0.0423803
8,-0.065282,1.248687e-06,True,2.69191e-05
9,0.084700,3.088177e-10,True,1.47083e-08
11,-0.069143,2.820378e-07,True,7.4053e-06
...,...,...,...,...
1984,0.040026,2.975246e-03,True,0.0185207
1989,0.036286,7.091784e-03,True,0.0347463
1991,0.039684,3.231006e-03,True,0.0198445
2009,0.034669,1.009781e-02,True,0.0446659


spiculation: 490


,correlation,pvalue,reject,adjusted
2,-0.034106,1.138342e-02,True,0.0175287
4,-0.059376,1.041649e-05,True,5.77132e-05
5,0.067968,4.473277e-07,True,5.14678e-06
8,-0.028687,3.330227e-02,True,0.0474952
11,-0.101275,5.003264e-14,True,2.49919e-12
...,...,...,...,...
2043,-0.046263,5.956883e-04,True,0.0013421
2044,-0.041563,2.039562e-03,True,0.00383565
2045,-0.044538,9.483533e-04,True,0.00200023
2046,-0.047991,3.681199e-04,True,0.000907232


subtlety: 1444
